In [1]:
from google.colab import drive

In [2]:
drive.mount("/F")

Mounted at /F


In [3]:
!ls '/F/My Drive/Maratona/Redes Neurais Artificiais/'

 census.csv				    house_prices.csv
'Classificação Binária XOR.ipynb'	   'Percepton de 1 Camada II.ipynb'
'Classificação Multiclasse Iris II.ipynb'  'Percepton de uma Camada.ipynb'
'Classificação MultiClasse Iris.ipynb'	    Untitled0.ipynb


In [4]:
import pandas as pd

In [5]:
base = pd.read_csv("/F/My Drive/Maratona/Redes Neurais Artificiais/house_prices.csv")

In [6]:
base.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [7]:
colunas_usadas = ['price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long']

In [9]:
base = pd.read_csv('/F/My Drive/Maratona/Redes Neurais Artificiais/house_prices.csv', usecols = colunas_usadas)

In [10]:
from sklearn.preprocessing import MinMaxScaler

In [11]:
scaler_x = MinMaxScaler()

In [12]:
base[['bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long']] = scaler_x.fit_transform(base[['bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long']])

In [13]:
base.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long
0,221900.0,0.090909,0.12500,0.067170,0.003108,0.0,0.0,0.0,0.5,0.500000,0.097588,0.000000,0.478261,0.000000,0.893939,0.571498,0.217608
1,538000.0,0.090909,0.28125,0.172075,0.004072,0.4,0.0,0.0,0.5,0.500000,0.206140,0.082988,0.443478,0.988089,0.626263,0.908959,0.166113
2,180000.0,0.060606,0.12500,0.036226,0.005743,0.0,0.0,0.0,0.5,0.416667,0.052632,0.000000,0.286957,0.000000,0.136364,0.936143,0.237542
3,604000.0,0.121212,0.37500,0.126038,0.002714,0.0,0.0,0.0,1.0,0.500000,0.083333,0.188797,0.565217,0.000000,0.681818,0.586939,0.104651
4,510000.0,0.090909,0.25000,0.104906,0.004579,0.0,0.0,0.0,0.5,0.583333,0.152412,0.000000,0.756522,0.000000,0.368687,0.741354,0.393688


In [14]:
scaler_y = MinMaxScaler()

In [15]:
base[['price']] = scaler_y.fit_transform(base[['price']])

In [16]:
base.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long
0,0.019266,0.090909,0.12500,0.067170,0.003108,0.0,0.0,0.0,0.5,0.500000,0.097588,0.000000,0.478261,0.000000,0.893939,0.571498,0.217608
1,0.060721,0.090909,0.28125,0.172075,0.004072,0.4,0.0,0.0,0.5,0.500000,0.206140,0.082988,0.443478,0.988089,0.626263,0.908959,0.166113
2,0.013770,0.060606,0.12500,0.036226,0.005743,0.0,0.0,0.0,0.5,0.416667,0.052632,0.000000,0.286957,0.000000,0.136364,0.936143,0.237542
3,0.069377,0.121212,0.37500,0.126038,0.002714,0.0,0.0,0.0,1.0,0.500000,0.083333,0.188797,0.565217,0.000000,0.681818,0.586939,0.104651
4,0.057049,0.090909,0.25000,0.104906,0.004579,0.0,0.0,0.0,0.5,0.583333,0.152412,0.000000,0.756522,0.000000,0.368687,0.741354,0.393688


In [17]:
X = base.drop('price', axis=1)

In [18]:
Y = base.price

In [19]:
X.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long
0,0.090909,0.12500,0.067170,0.003108,0.0,0.0,0.0,0.5,0.500000,0.097588,0.000000,0.478261,0.000000,0.893939,0.571498,0.217608
1,0.090909,0.28125,0.172075,0.004072,0.4,0.0,0.0,0.5,0.500000,0.206140,0.082988,0.443478,0.988089,0.626263,0.908959,0.166113
2,0.060606,0.12500,0.036226,0.005743,0.0,0.0,0.0,0.5,0.416667,0.052632,0.000000,0.286957,0.000000,0.136364,0.936143,0.237542
3,0.121212,0.37500,0.126038,0.002714,0.0,0.0,0.0,1.0,0.500000,0.083333,0.188797,0.565217,0.000000,0.681818,0.586939,0.104651
4,0.090909,0.25000,0.104906,0.004579,0.0,0.0,0.0,0.5,0.583333,0.152412,0.000000,0.756522,0.000000,0.368687,0.741354,0.393688


In [21]:
Y.head()

0    0.019266
1    0.060721
2    0.013770
3    0.069377
4    0.057049
Name: price, dtype: float64

In [22]:
previsores_colunas = colunas_usadas[1:17]

In [23]:
import tensorflow as tf

In [24]:
colunas = [tf.feature_column.numeric_column(key = c) for c in previsores_colunas]

In [25]:
from sklearn.model_selection import train_test_split

In [27]:
X_treinamento, X_teste, Y_treinamento, Y_teste = train_test_split(X, Y, test_size = 0.3, random_state = 42)

In [28]:
X_treinamento.shape

(15129, 16)

In [29]:
X_teste.shape

(6484, 16)

In [30]:
def make_input_fn(X, y = None, num_epochs = None, shuffle = True, batch_size = 32):
    def input_function():
        ds = tf.data.Dataset.from_tensor_slices((dict(X), y))
        if shuffle:
            ds = ds.shuffle(1000)
        ds = ds.batch(batch_size).repeat(num_epochs)
        return ds
    return input_function

In [32]:
funcao_treinamento = make_input_fn(X_treinamento, Y_treinamento)

In [33]:
config = tf.estimator.RunConfig(tf_random_seed = 24)
regressor = tf.estimator.DNNRegressor(hidden_units = [8, 8, 8], feature_columns = colunas, config = config)
regressor.train(input_fn = funcao_treinamento, steps = 20000)

INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp9p05je66', '_tf_random_seed': 24, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and grap

In [34]:
funcao_previsao = make_input_fn(X_teste, num_epochs = 1, shuffle = False)

In [35]:
funcao_previsao = make_input_fn(X_teste, num_epochs = 1, shuffle = False)

In [36]:
previsoes = regressor.predict(input_fn = funcao_previsao)
list(previsoes)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp9p05je66/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[{'predictions': array([0.04751214], dtype=float32)},
 {'predictions': array([0.06648014], dtype=float32)},
 {'predictions': array([0.10735314], dtype=float32)},
 {'predictions': array([0.11447291], dtype=float32)},
 {'predictions': array([0.06656931], dtype=float32)},
 {'predictions': array([0.03041746], dtype=float32)},
 {'predictions': array([0.05897974], dtype=float32)},
 {'predictions': array([0.07188579], dtype=float32)},
 {'predictions': array([0.0506176], dtype=float32)},
 {'predictions': array([0.05551919], dtype=float32)},
 {'predictions': array([0.07061736], dtype=float32)},
 {'predictions': array([0.08183684], dtype=float32)},
 {'predictions': array([0.04513727], dtype=float32)},
 {'predictions': array([0.05789351], dtype=float32)},
 {'predictions': array([0.05564658], dtype=float32)},
 {'predictions': array([0.07043485], dtype=float32)},
 {'predictions': array([0.04580815], dtype=float32)},
 {'predictions': array([0.07294612], dtype=float32)},
 {'predictions': array([0.051

In [37]:
valores_previsao = []
for p in regressor.predict(input_fn = funcao_previsao):
    valores_previsao.append(p['predictions'][0])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp9p05je66/model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [38]:
valores_previsao

[0.047512136,
 0.066480145,
 0.10735314,
 0.11447291,
 0.06656931,
 0.03041746,
 0.058979742,
 0.07188579,
 0.050617598,
 0.055519193,
 0.07061736,
 0.08183684,
 0.04513727,
 0.057893507,
 0.055646583,
 0.070434846,
 0.04580815,
 0.072946124,
 0.051990382,
 0.056527242,
 0.06276323,
 0.0916877,
 0.06441228,
 0.10469201,
 0.07001004,
 0.062677026,
 0.045842916,
 0.03934589,
 0.052214473,
 0.066097595,
 0.03277471,
 0.07210102,
 0.03868389,
 0.037055798,
 0.043339744,
 0.05925125,
 0.0581394,
 0.05201614,
 0.045581006,
 0.06296748,
 0.056385122,
 0.06292592,
 0.03450793,
 0.049297236,
 0.0502062,
 0.036614783,
 0.03630638,
 0.04885052,
 0.043757275,
 0.07821996,
 0.06808466,
 0.08555067,
 0.074886456,
 0.06398998,
 0.06952497,
 0.08339715,
 0.035451464,
 0.06769628,
 0.04527448,
 0.068272986,
 0.080105536,
 0.047797263,
 0.0631008,
 0.10465265,
 0.062958,
 0.010512808,
 0.04721839,
 0.34614766,
 0.050665967,
 0.07316055,
 0.039036967,
 0.045763046,
 0.083348855,
 0.048742585,
 0.06752093

In [39]:
import numpy as np
valores_previsao = np.asarray(valores_previsao).reshape(-1, 1)
valores_previsao = scaler_y.inverse_transform(valores_previsao)
valores_previsao

array([[437280.06],
       [581911.06],
       [893567.7 ],
       ...,
       [578237.06],
       [794145.2 ],
       [595309.8 ]], dtype=float32)

In [41]:
y_teste2 = Y_teste.values.reshape(-1, 1)
y_teste2 = scaler_y.inverse_transform(y_teste2)
y_teste2

array([[ 365000.],
       [ 865000.],
       [1038000.],
       ...,
       [ 410500.],
       [ 940000.],
       [ 410000.]])

In [42]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_teste2, valores_previsao)
mae

199416.1734244872